<h1 style="text-align:center;vertical-align:middle">Youtube数据分析与预处理报告</h1>
## 数据概览
数据给出了不同国家在Youtube上的流行视频的观看情况。

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
def load_data(path,filename):
    return pd.read_csv(path + '/' + filename)

In [5]:
root = '../Assignment4/youtube-new'
CA_data = load_data(root,'CAvideos.csv')
CA_data.columns

Index(['video_id', 'trending_date', 'title', 'channel_title', 'category_id',
       'publish_time', 'tags', 'views', 'likes', 'dislikes', 'comment_count',
       'thumbnail_link', 'comments_disabled', 'ratings_disabled',
       'video_error_or_removed', 'description'],
      dtype='object')

每行数据包括个列，分别代表：

- `video_id` 视频id，唯一属性
- `trending_date` 流行时间
- `title` 视频标题
- `channel_title` 发布频道
- `category_id` 类别id（每个国家对应的类别不同）
- `publish_time` 发布时间
- `tags` 视频标签
- `views` 观看数量
- `likes` 喜欢数量
- `dislikes` 不喜欢数量
- `comment_count` 评论数量
- `thumbnail_link` 视频链接，唯一属性
- `comments_disabled` 是否禁止评论
- `ratings_disabled` 是否禁止打分
- `video_error_or_removed` 是否错误或删除
- `description` 视频描述

接下来的内容将区分国家，对以上属性中的数值属性以及有意义的非唯一属性进行数据分析与预处理。

## 1 中国(CA)
### 1.1 流行时间trending_date(无缺失值)
**频率统计**

In [36]:
freq = CA_data['trending_date'].value_counts()
freq

17.08.12    200
18.06.04    200
17.26.12    200
18.05.01    200
18.08.06    200
           ... 
18.13.02    195
18.02.02    195
18.31.01    191
18.01.02    190
18.14.05    171
Name: trending_date, Length: 205, dtype: int64

In [41]:
for i in set(freq.values):
    print(i,end=':')
    for j in list(filter(lambda x : freq[x] == i,sorted(freq.index))):
        print(j,end=' ')
    print()

195:18.02.02 18.13.02 
196:18.29.01 
197:18.09.03 18.19.03 18.25.04 18.28.02 
198:17.07.12 18.06.02 18.15.02 18.19.05 18.23.02 18.24.02 18.27.02 
199:17.04.12 17.12.12 17.19.11 17.21.12 17.26.11 17.29.12 18.03.02 18.03.03 18.04.03 18.05.02 18.07.03 18.10.02 18.10.05 18.11.02 18.14.02 18.14.03 18.14.04 18.15.05 18.16.02 18.17.02 18.18.02 18.19.01 18.21.02 18.22.01 18.22.02 18.23.04 18.26.02 18.27.01 18.27.03 18.28.03 18.29.04 
200:17.01.12 17.02.12 17.03.12 17.05.12 17.06.12 17.08.12 17.09.12 17.10.12 17.11.12 17.13.12 17.14.11 17.14.12 17.15.11 17.15.12 17.16.11 17.16.12 17.17.11 17.17.12 17.18.11 17.18.12 17.19.12 17.20.11 17.20.12 17.21.11 17.22.11 17.22.12 17.23.11 17.23.12 17.24.11 17.24.12 17.25.11 17.25.12 17.26.12 17.27.11 17.27.12 17.28.11 17.28.12 17.29.11 17.30.11 17.30.12 17.31.12 18.01.01 18.01.03 18.01.04 18.01.05 18.01.06 18.02.01 18.02.03 18.02.04 18.02.05 18.02.06 18.03.01 18.03.04 18.03.05 18.03.06 18.04.01 18.04.02 18.04.04 18.04.05 18.04.06 18.05.01 18.05.03 18.05.04

由以上结果可知（左边为出现频数，右边为日期），大多数流行时间的trending viedo的个数都为200个，其余少数日期的个数为171、190、191、195、196、197、198、199，无缺失值。

**频数分布**